In [1]:
# Packages 
import os
os.chdir('../')

import time
start_time = time.time()

import numpy as np
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import matplotlib
from matplotlib import pyplot as plt
from nilearn import datasets
from nilearn import plotting
import nibabel as nib
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision.models import AlexNet_Weights, VGG16_Weights, VGG16_BN_Weights, VGG19_Weights, EfficientNet_B2_Weights
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LinearRegression, Ridge
from scipy.stats import pearsonr as corr

# Functions and classes
# Platform definition
platform = 'jupyter_notebook'
if platform == 'jupyter_notebook':
    # data_dir = '../../../Projects/Datasets/Biomedical/algonauts_2023_challenge_data'
    # Data folder definition
    data_dir = '../Datasets/Biomedical/algonauts_2023_challenge_data'
    # Used to save the prediction of saved model
    parent_submission_dir = './algonauts_2023_challenge_submission'
    ncsnr_dir = '../Datasets/Biomedical/algonauts_ncsnr'
    images_trials = '../Datasets/Biomedical/algonauts_train_images_trials'
        
class argObj:
  def __init__(self, data_dir, parent_submission_dir, subj, parent_ncsnr_dir = ncsnr_dir, images_trials_parent_dir = images_trials):
    # Define the dir where data is stored
    # 1 became 01
    self.subj = format(subj, '02') # '0numberofchars'
    self.data_dir = os.path.join(data_dir, 'subj'+self.subj)

    # NCSNR
    self.parent_ncsnr_dir = parent_ncsnr_dir
    self.ncsnr_dir = os.path.join(self.parent_ncsnr_dir, 'subj'+self.subj)
    
    # SUBMISSION DIR
    self.parent_submission_dir = parent_submission_dir
    self.subject_submission_dir = os.path.join(self.parent_submission_dir,
        'subj'+self.subj)
    # Create the submission directory if not existing
    if not os.path.isdir(self.subject_submission_dir):
        os.makedirs(self.subject_submission_dir)

    # Train Images Trials 
    self.images_trials_parent_dir = images_trials_parent_dir
    self.images_trials_dir = os.path.join(self.images_trials_parent_dir, 'subj'+self.subj)

In [2]:
### Parameters ###
pca_component = 300
train_percentage = 90 # X% of the training data will be used for training, (100-X)% for validation
batch_mode = "static" # "dynamic" or "static"
batch_size_min = 40 # Batch size has to be major than pca_component
batch_size_max = 100
batch_size = 200
save_predictions = True 
feature_model_type = "alexnet" #@param ["alexnet", "vgg16", "efficientnetb2", "efficientnetb2lib"]
regression_type = "linear" #@param ["linear", "ridge"]

if __name__ == "__main__":
            
    # Select the device to run the model on
    device = 'cuda' #@param ['cpu', 'cuda'] {allow-input: true}
    device = torch.device(device)
    
    # Check if GPU is available and torch is using it
    print("Check if GPU is available and if torch is using it ..")
    print("\n")
    print("Torch Cuda is available?")
    print(torch.cuda.is_available())
    print("Torch Cuda device count is :")
    print(torch.cuda.device_count())
    print("Torch Cuda current device is :")
    print(torch.cuda.current_device())
    print("Torch Cuda device is :")
    print(torch.cuda.device(0))
    print(torch.cuda.get_device_name(0))
    print("Pytorch version：")
    print(torch.__version__)
    print("CUDA Version: ")
    print(torch.version.cuda)
    print("cuDNN version is :")
    print(torch.backends.cudnn.version())
    print("\n")

Check if GPU is available and if torch is using it ..


Torch Cuda is available?
True
Torch Cuda device count is :
1
Torch Cuda current device is :
0
Torch Cuda device is :
NVIDIA GeForce RTX 3070 Laptop GPU
Pytorch version：
1.13.0
CUDA Version: 
11.6
cuDNN version is :
8302




In [3]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

0
0


In [4]:
subj = 1
args = argObj(data_dir, parent_submission_dir, subj)
        
## Load the fmri response for left and right hemispheres ##
fmri_dir = os.path.join(args.data_dir, 'training_split', 'training_fmri')
lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))
# Check the shapes of the data
print('LH training fMRI data shape:', lh_fmri.shape)
print('RH training fMRI data shape:', rh_fmri.shape)

## load the stimulus images ##
train_img_dir  = os.path.join(args.data_dir, 'training_split', 'training_images')
test_img_dir  = os.path.join(args.data_dir, 'test_split', 'test_images')
# Create lists will all training and test image file names, sorted
train_img_list = os.listdir(train_img_dir)
train_img_list.sort()
test_img_list = os.listdir(test_img_dir)
test_img_list.sort()
        
## Create the training, validation and test partitions indices ##
rand_seed = 5 #@param
np.random.seed(rand_seed)
# train_percentage = 90
# Calculate how many stimulus images correspond to 90% of the training data
num_train = int(np.round(len(train_img_list) / 100 * train_percentage))
# Shuffle all training stimulus images
idxs = np.arange(len(train_img_list))
np.random.shuffle(idxs)
# Assign 90% of the shuffled stimulus images to the training partition,
# and 10% to the test partition
idxs_train, idxs_val = idxs[:num_train], idxs[num_train:]
# No need to shuffle or split the test stimulus images
idxs_test = np.arange(len(test_img_list))
print('Total train images: ' + str(len(train_img_list)))
print('Training stimulus images: ' + format(len(idxs_train)))
print('Validation stimulus images: ' + format(len(idxs_val)))
print('Test stimulus images: ' + format(len(idxs_test)))

## Create the training, validation and test partitions dataloaders ##
# Preprocessing pipeline
transform = transforms.Compose([
    transforms.Resize((256,256)), # resize the images to 224x224 pixels (256x256)
    transforms.CenterCrop(224), # IN THE OLDER SCRIPT: no center crop, only resize to 224x224
    transforms.ToTensor(), # convert the images to a PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    # normalize the images color channels
    # mean: [0.485, 0.456, 0.406] for the three channels
    # std: [0.229, 0.224, 0.225] for the three channels
])
# Dataset class
class ImageDataset(Dataset):
    def __init__(self, imgs_paths, idxs, transform):
        self.imgs_paths = np.array(imgs_paths)[idxs]
        self.transform = transform

    def __len__(self):
        return len(self.imgs_paths)

    def __getitem__(self, idx):
        # Load the image
        img_path = self.imgs_paths[idx]
        img = Image.open(img_path).convert('RGB')
        # Preprocess the image and send it to the chosen device ('cpu' or 'cuda')
        if self.transform:
            img = self.transform(img)
        return img
# Dataloader class
# 3 6 -> 310 / other ones -> 300
# if batch_mode == 'static':
#     if subj == 3 or subj == 6:
#         batch_size = 310
#     else: 
#         batch_size = 300 # 300 #@param (310 for 3 and 6)
#     print(batch_mode, "batch size: ", batch_size)
# if batch_mode == 'dynamic':
#     # Batch size should never be less than pca components
#     for batch_size in range(batch_size_min, batch_size_max + 1):
#         if len(idxs_train) % batch_size >= batch_size_min:
#             # print("Train images: ", len(train_img_list))
#             # print("Remainder: ", len(train_img_list) % batch_size)
#             print(batch_mode, "Batch size: ", batch_size)
#             break

# Get the paths of all image files
train_imgs_paths = sorted(list(Path(train_img_dir).iterdir()))
test_imgs_paths = sorted(list(Path(test_img_dir).iterdir()))

# The DataLoaders contain the ImageDataset class
train_imgs_dataloader = DataLoader(
    ImageDataset(train_imgs_paths, idxs_train, transform), 
    batch_size=batch_size
)
val_imgs_dataloader = DataLoader(
    ImageDataset(train_imgs_paths, idxs_val, transform), 
    batch_size=batch_size
)
test_imgs_dataloader = DataLoader(
    ImageDataset(test_imgs_paths, idxs_test, transform), 
    batch_size=batch_size
)

## Spli the fmri data into training and validation partitions ##
lh_fmri_train = lh_fmri[idxs_train]
lh_fmri_val = lh_fmri[idxs_val]
rh_fmri_train = rh_fmri[idxs_train]
rh_fmri_val = rh_fmri[idxs_val]
del lh_fmri, rh_fmri


LH training fMRI data shape: (9841, 19004)
RH training fMRI data shape: (9841, 20544)
Total train images: 9841
Training stimulus images: 8857
Validation stimulus images: 984
Test stimulus images: 159


In [5]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

0
0


In [6]:
if feature_model_type == 'alexnet':
    ## AlexNet ##
    model = torch.hub.load('pytorch/vision:v0.10.0', 
                        'alexnet', 
                        weights=AlexNet_Weights.IMAGENET1K_V1)
    model.to(device) # send the model to the chosen device ('cpu' or 'cuda')
    model.eval() # set the model to evaluation mode, since you are not training it
    train_nodes, _ = get_graph_node_names(model)
    model_layer = "features.12" #@param ["features.2", "features.5", "features.7", "features.9", "features.12", "classifier.2", "classifier.5", "classifier.6"] {allow-input: true}
    feature_extractor = create_feature_extractor(model, return_nodes=[model_layer])
    for param in feature_extractor.parameters():
        param.requires_grad = False
elif feature_model_type == 'vgg16':
    model = torch.hub.load('pytorch/vision:v0.10.0', 
                        'vgg16_bn', 
                        weights=VGG16_BN_Weights.IMAGENET1K_V1)
    model.eval() # set the model to evaluation mode, since you are not training it
    model.to(device) # send the model to the chosen device ('cpu' or 'cuda')
    model_layer = "flatten" #@param ["features.2", "features.5", "features.7", "features.10", "features.12", "features.14", "features.17", "features.19", "features.21", "features.24", "features.26", "features.28", "features.31", "features.33", "features.35", "classifier.2", "classifier.5", "classifier.7"] {allow-input: true}
    feature_extractor = create_feature_extractor(model, return_nodes=[model_layer])
    # Freeze the parameters of the feature extractor
    for param in feature_extractor.parameters():
        param.requires_grad = False
elif feature_model_type == 'efficientnetb2':
    model = models.efficientnet_b2(weights = EfficientNet_B2_Weights.IMAGENET1K_V1)
    model.eval() # set the model to evaluation mode, since you are not training it
    model.to(device) # send the model to the chosen device ('cpu' or 'cuda')
    model_layer = "features.8" #@param ['x', 'features.0', 'features.1.0.block.0', 'features.1.0.block.1', 'features.1.0.block.2', 'features.1.1.block.0', 'features.1.1.block.1', 'features.1.1.block.2', 'features.1.1.stochastic_depth', 'features.1.1.add', 'features.2.0.block.0', 'features.2.0.block.1', 'features.2.0.block.2', 'features.2.0.block.3', 'features.2.1.block.0', 'features.2.1.block.1', 'features.2.1.block.2', 'features.2.1.block.3', 'features.2.1.stochastic_depth', 'features.2.1.add', 'features.2.2.block.0', 'features.2.2.block.1', 'features.2.2.block.2', 'features.2.2.block.3', 'features.2.2.stochastic_depth', 'features.2.2.add', 'features.3.0.block.0', 'features.3.0.block.1', 'features.3.0.block.2', 'features.3.0.block.3', 'features.3.1.block.0', 'features.3.1.block.1', 'features.3.1.block.2', 'features.3.1.block.3', 'features.3.1.stochastic_depth', 'features.3.1.add', 'features.3.2.block.0', 'features.3.2.block.1', 'features.3.2.block.2', 'features.3.2.block.3', 'features.3.2.stochastic_depth', 'features.3.2.add', 'features.4.0.block.0', 'features.4.0.block.1', 'features.4.0.block.2', 'features.4.0.block.3', 'features.4.1.block.0', 'features.4.1.block.1', 'features.4.1.block.2', 'features.4.1.block.3', 'features.4.1.stochastic_depth', 'features.4.1.add', 'features.4.2.block.0', 'features.4.2.block.1', 'features.4.2.block.2', 'features.4.2.block.3', 'features.4.2.stochastic_depth', 'features.4.2.add', 'features.4.3.block.0', 'features.4.3.block.1', 'features.4.3.block.2', 'features.4.3.block.3', 'features.4.3.stochastic_depth', 'features.4.3.add', 'features.5.0.block.0', 'features.5.0.block.1', 'features.5.0.block.2', 'features.5.0.block.3', 'features.5.1.block.0', 'features.5.1.block.1', 'features.5.1.block.2', 'features.5.1.block.3', 'features.5.1.stochastic_depth', 'features.5.1.add', 'features.5.2.block.0', 'features.5.2.block.1', 'features.5.2.block.2', 'features.5.2.block.3', 'features.5.2.stochastic_depth', 'features.5.2.add', 'features.5.3.block.0', 'features.5.3.block.1', 'features.5.3.block.2', 'features.5.3.block.3', 'features.5.3.stochastic_depth', 'features.5.3.add', 'features.6.0.block.0', 'features.6.0.block.1', 'features.6.0.block.2', 'features.6.0.block.3', 'features.6.1.block.0', 'features.6.1.block.1', 'features.6.1.block.2', 'features.6.1.block.3', 'features.6.1.stochastic_depth', 'features.6.1.add', 'features.6.2.block.0', 'features.6.2.block.1', 'features.6.2.block.2', 'features.6.2.block.3', 'features.6.2.stochastic_depth', 'features.6.2.add', 'features.6.3.block.0', 'features.6.3.block.1', 'features.6.3.block.2', 'features.6.3.block.3', 'features.6.3.stochastic_depth', 'features.6.3.add', 'features.6.4.block.0', 'features.6.4.block.1', 'features.6.4.block.2', 'features.6.4.block.3', 'features.6.4.stochastic_depth', 'features.6.4.add', 'features.7.0.block.0', 'features.7.0.block.1', 'features.7.0.block.2', 'features.7.0.block.3', 'features.7.1.block.0', 'features.7.1.block.1', 'features.7.1.block.2', 'features.7.1.block.3', 'features.7.1.stochastic_depth', 'features.7.1.add', 'features.8', 'avgpool', 'flatten', 'classifier.0', 'classifier.1']
    feature_extractor = create_feature_extractor(model, return_nodes=[model_layer])
    for param in feature_extractor.parameters():
        param.requires_grad = False
    print(f'Feature extractor: {feature_model_type}, layer: {model_layer}')
elif feature_model_type == 'efficientnetb2lib':
    model = EfficientNet.from_pretrained('efficientnet-b2')
    model = nn.Sequential(*list(model.children())[:-1])
    model.eval()
    for param in model.parameters():
        param.requires_grad = False
    def feature_extractor(x):
        with torch.no_grad():
            x = model(x)
            return x

Using cache found in C:\Users\giorg/.cache\torch\hub\pytorch_vision_v0.10.0


In [6]:
train_nodes, _ = get_graph_node_names(model)
print(train_nodes)

['x', 'features.0', 'features.1', 'features.2', 'features.3', 'features.4', 'features.5', 'features.6', 'features.7', 'features.8', 'features.9', 'features.10', 'features.11', 'features.12', 'features.13', 'features.14', 'features.15', 'features.16', 'features.17', 'features.18', 'features.19', 'features.20', 'features.21', 'features.22', 'features.23', 'features.24', 'features.25', 'features.26', 'features.27', 'features.28', 'features.29', 'features.30', 'features.31', 'features.32', 'features.33', 'features.34', 'features.35', 'features.36', 'features.37', 'features.38', 'features.39', 'features.40', 'features.41', 'features.42', 'features.43', 'avgpool', 'flatten', 'classifier.0', 'classifier.1', 'classifier.2', 'classifier.3', 'classifier.4', 'classifier.5', 'classifier.6']


In [7]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

244797440
257949696


In [8]:
torch.cuda.empty_cache()

In [9]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

244797440
257949696


Il modello va prima spostato alla cpu prima di essere eliminato (pulizia gpu)

In [10]:
# model.to('cpu')
# del model
# torch.cuda.empty_cache()

pulizia memoria ram

In [6]:
# import gc
# gc.collect()

513

In [48]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

0
0


In [27]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

0
0


# Incremental PCA + Feature Extraction Fit

## Old One (batch_size = pca_batch_size)

In [ ]:
## PCA ##
def fit_pca(feature_extractor, dataloader):

    # Define PCA parameters
    pca = IncrementalPCA(n_components=pca_component, batch_size=batch_size)

    with torch.no_grad(): 
        # Fit PCA to batch
        for _, d in tqdm(enumerate(dataloader), total=len(dataloader)):
            # Extract features
            ft = feature_extractor(d.to(device))
            # Flatten the features
            ft = torch.hstack([torch.flatten(l, start_dim=1) for l in ft.values()])
            # Fit PCA to batch
            print(ft.shape)
            pca.partial_fit(ft.detach().cpu().numpy())
            torch.cuda.empty_cache()
        return pca

In [10]:
## Feature extraction ##
def extract_features(feature_extractor, dataloader):
    features = []
    for _, d in tqdm(enumerate(dataloader), total=len(dataloader)):
        # Extract features
        ft = feature_extractor(d.to(device))
        # Flatten the features
        ft = torch.hstack([torch.flatten(l, start_dim=1) for l in ft.values()])
        # Apply PCA transform
        # print(ft.shape)
        features.append(ft.detach().cpu().numpy())
    return np.vstack(features)
    
print('Extracting features from training, validation and test data...')
features_train = extract_features(feature_extractor, train_imgs_dataloader)
features_val = extract_features(feature_extractor, val_imgs_dataloader)
features_test = extract_features(feature_extractor, test_imgs_dataloader)
print('Features shape: ', features_train.shape)
print('Features shape: ', features_val.shape)
print('Features shape: ', features_test.shape)

Extracting features from training, validation and test data...


100%|██████████| 1/1 [00:02<00:00,  2.08s/it]

Features shape:  (8857, 9216)
Features shape:  (984, 9216)
Features shape:  (159, 9216)


## New One (batch_size != pca_batch_size)

### PCA Batch Size calculator

Batch size calculator:
Code that allows you to find the value of the variable 'n_stacked_batches' given that:

- The variables total_instances (e.g. equal to 9000), batch_size (e.g. equal to 64), and pca_component (e.g. equal to 200) are known.
- min_pca_batch_size = pca_component * 2
- It is known that pca_batch_size = batch_size * n_stacked_batches
It find the value of n_stacked_batches so that:
1. total_instances % pca_batch_size >= min_pca_batch_size
2. n_stacked_batches must be the smallest available value that satisfies this inequality.

In [9]:
total_instances = 8431 # len(idxs_train)
batch_size = 64
pca_component = 500

min_pca_batch_size = pca_component * 2

n_stacked_batches = 1
while True:
    pca_batch_size = batch_size * n_stacked_batches
    if total_instances % pca_batch_size >= min_pca_batch_size:
        break
    n_stacked_batches += 1
pca_batch_size = batch_size * n_stacked_batches

print(f'Batches size: {batch_size}')
print(f'Total instances: {total_instances}')
print(f'PCA components: {pca_component}')
print(f'Minimum pca batch size: {min_pca_batch_size}')
print(f'Number of stacked batches for pca: {n_stacked_batches}')
print(f'PCA batch size (batch_size * n_stacked_batches): {pca_batch_size}')
print(f'Last pca batch size: {total_instances % pca_batch_size}')


Batches size: 64
Total instances: 8431
PCA components: 500
Minimum pca batch size: 1000
Number of stacked batches for pca: 19
PCA batch size (batch_size * n_stacked_batches): 1216
Last pca batch size: 1135


### Feature Extractor + Incremental PCA FIT

The feature extractor + pca fit function is able to:
1. extract features batches from images batches, obtaining 2d numpy matrices (batch_size x n_features) 
2. stack n_stacked_batches feature batches vertically
3. apply pca partial fit on stacked features

#### Test

In [ ]:
from tqdm import tqdm
import numpy as np
# Initialize empty ft_stacked matrix
ft_stacked = None
# Initialize counter for stacked batches
count_stacked_batches = 0
n = 122
array_di_batchsize = np.full((n), 64).reshape(n)

for _, d in enumerate(array_di_batchsize):
    print(_)
    # Stack vertically the ft matrix
    if ft_stacked is None:
        print(f"Inizio stacking bacth 1 di {n_stacked_batches}")
        ft_stacked = 1
    else:
        print("Stacking batch numero: ", count_stacked_batches + 1, " di ", n_stacked_batches)
        ft_stacked = 1
        
    # Check if n_stacked_batches is reached
    count_stacked_batches += 1
    if count_stacked_batches == n_stacked_batches:
        # Do something with ft_stacked (e.g., save to disk)
        # ...

        # Reset ft_stacked and counter
        print(f"Resetto ft_stacked e count_stacked_batches a 0")
        ft_stacked = None
        count_stacked_batches = 0

# Check if there are remaining stacked batches to process
if count_stacked_batches > 0:
    # Do something with ft_stacked (e.g., save to disk)
    # ...
    print(f"Ultimo batch di ft_stacked incompleto con {count_stacked_batches} batch su {n_stacked_batches}")


#### Final PCA Fit function

In [ ]:
def fit_pca(feature_extractor, dataloader, pca_component, pca_batch_size):
    # Initialize empty ft_stacked matrix which will be used to stack the features 
    # of n_stacked_batches batches
    ft_stacked = None
    # Initialize counter for stacked batches
    count_stacked_batches = 0
    # Define PCA parameters
    pca = IncrementalPCA(n_components=pca_component, batch_size=pca_batch_size)

    with torch.no_grad(): 
        # Fit PCA to batch
        for _, d in tqdm(enumerate(dataloader), total=len(dataloader)):
            # Bove batch to gpu and extract features
            ft = feature_extractor(d.to(device))
            # Flatten the features and detach them from the graph
            ft = torch.hstack([torch.flatten(l, start_dim=1) for l in ft.values()]).detach().cpu().numpy()
            # Fit PCA to batch
            print(ft.shape)
            # Stack vertically the ft matrix
            if ft_stacked is None:
                # If the feature batch is the first one, initialize ft_stacked
                ft_stacked = ft
            else:
                # if the feature batch is not the first one, stack vertically the ft matrix
                ft_stacked = np.vstack((ft_stacked, ft))
            # Check if n_stacked_batches is reached
            count_stacked_batches += 1
            if count_stacked_batches == n_stacked_batches:
                print(ft_stacked.shape)
                # If n_stacked_batches is reached, fit PCA to ft_stacked
                pca.partial_fit(ft_stacked)
                # After fitting PCA, reset ft_stacked and counter
                ft_stacked = None
                count_stacked_batches = 0
            # Free VRAM memory by deleting model graph
            torch.cuda.empty_cache()
        # Check if there are remaining stacked batches to process
        # that will be the case if the total number of batches is not a multiple of n_stacked_batches
        # the last pca batch will be smaller than pca_batch_size
        if count_stacked_batches > 0:
            # Fit PCA to ft_stacked
            pca.partial_fit(ft_stacked)
            torch.cuda.empty_cache()
        return pca

In [ ]:
## Fit PCA to training data ##
print(f'Fitting Incremental PCA ({pca_component} components) to training data...')
pca = fit_pca(feature_extractor, train_imgs_dataloader)
print("Comulative Explained variance ratio: ", sum(pca.explained_variance_ratio_))
print("Number of components: ", pca.n_components_)

#### Final feature extraction function 

In [ ]:
def extract_features(feature_extractor, dataloader, pca):
    # Initialize empty ft_stacked matrix which will be used to stack the features 
    # of n_stacked_batches batches
    ft_stacked = None
    # Initialize counter for stacked batches
    count_stacked_batches = 0
    # Define list to store downsampled features
    features = []

    # Set model to evaluation mode (e.g., disable dropout)
    with torch.no_grad(): 
        for _, d in tqdm(enumerate(dataloader), total=len(dataloader)):
            # Bove batch to gpu and extract features
            ft = feature_extractor(d.to(device))
            # Flatten the features and detach them from the graph
            ft = torch.hstack([torch.flatten(l, start_dim=1) for l in ft.values()]).detach().cpu().numpy()
            # Fit PCA to batch
            print(ft.shape)
            # Stack vertically the ft matrix
            if ft_stacked is None:
                # If the feature batch is the first one, initialize ft_stacked
                ft_stacked = ft
            else:
                # if the feature batch is not the first one, stack vertically the ft matrix
                ft_stacked = np.vstack((ft_stacked, ft))
            # Check if n_stacked_batches is reached
            count_stacked_batches += 1
            if count_stacked_batches == n_stacked_batches:
                print(ft_stacked.shape)
                # If n_stacked_batches is reached, transform ft_stacked with PCA
                ft_stacked = pca.transform(ft_stacked)
                # Append downsampled features to features list
                features.append(ft_stacked)
                # After transforming with PCA, reset ft_stacked and counter
                ft_stacked = None
                count_stacked_batches = 0
            # Free VRAM memory by deleting model graph
            torch.cuda.empty_cache()
        # Check if there are remaining stacked batches to process
        # that will be the case if the total number of batches is not a multiple of n_stacked_batches
        # the last pca batch will be smaller than pca_batch_size
        if count_stacked_batches > 0:
            # Fit PCA to ft_stacked
            ft_stacked = pca.transform(ft_stacked)
            torch.cuda.empty_cache()
        return pca

In [ ]:
print('Extracting features from training, validation and test data...')
features_train = extract_features(feature_extractor, train_imgs_dataloader, pca)
features_val = extract_features(feature_extractor, val_imgs_dataloader, pca)
features_test = extract_features(feature_extractor, test_imgs_dataloader, pca)
del model, pca, feature_extractor

## VRAM, RAM and CACHE cleaning

In [11]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

244797440
799014912


In [15]:
torch.cuda.empty_cache()

In [12]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

244797440
799014912


In [13]:
import gc
model.to('cpu') # sposto sulla ram
feature_extractor.to('cpu') # sposto sulla ram
del model, feature_extractor, pca # elimino dalla ram
torch.cuda.empty_cache() # elimino la chache vram
gc.collect() # elimino la cache ram

AlexNet(
  (features): Module(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)

In [16]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

0
0


In [ ]:
del model, feature_extractor
torch.cuda.empty_cache()

In [ ]:
del feature_extractor

In [29]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

1024
2097152


In [9]:
pca_batch_size = 1000
pca = IncrementalPCA(n_components=pca_component, batch_size=pca_batch_size).fit(features_train)
print("Comulative Explained variance ratio: ", sum(pca.explained_variance_ratio_))
print("Number of components: ", pca.n_components_)

Comulative Explained variance ratio:  0.5664719757071206
Number of components:  300


In [10]:
from sklearn.decomposition import PCA
pca = PCA(n_components=pca_component).fit(features_train)
print("Comulative Explained variance ratio: ", sum(pca.explained_variance_ratio_))
print("Number of components: ", pca.n_components_)

Comulative Explained variance ratio:  0.5763521306798793
Number of components:  300
